In [1]:
import os, glob, json
import pickle, dill
import pandas as pd
pd.set_option('display.max_colwidth', None)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import numpy as np
from collections import Counter, defaultdict
from tqdm import tqdm, trange

In [3]:
# Load argument paths dataframe
per_path_df = pd.read_csv('per_path_df.csv')
for col in ['path_root_to_leaf','filtered_path_root_to_leaf']:
    per_path_df[col] = per_path_df[col].apply(lambda x: json.loads(x))
print(per_path_df.shape)
display(per_path_df.head())
#per_path_df.loc[0]['filtered_path_root_to_leaf'][0]

(528531, 11)


,subtree_root_id,subtree_leaf_id,entry_order,degree_back_and_forth,convo_delta_pct,won_delta,convo_id,len,path_root_to_leaf,filtered_path_root_to_leaf,guid
0,t1_e45llf2,t1_e476c6z,19,2,0.052632,1,t3_973yb1,190.25,"[t1_e45llf2, t1_e45mtzr, t1_e45p1ij, t1_e45pmfu, t1_e45q2fr, t1_e45qn42, t1_e46iqqp, t1_e476c6z]","[t1_e45llf2, t1_e45mtzr, t1_e45p1ij, t1_e45pmfu, t1_e45q2fr, t1_e45qn42, t1_e46iqqp, t1_e476c6z]",t1_e45llf2 | t1_e476c6z
1,t1_e45lq41,t1_e45o2sv,20,3,0.052632,1,t3_973yb1,125.25,"[t1_e45lq41, t1_e45n24e, t1_e45nh0b, t1_e45o2sv]","[t1_e45lq41, t1_e45n24e, t1_e45nh0b, t1_e45o2sv]",t1_e45lq41 | t1_e45o2sv
2,t1_e45llf2,t1_e47qnva,19,2,0.052632,1,t3_973yb1,135.60,"[t1_e45llf2, t1_e45mtzr, t1_e45p1ij, t1_e45pmfu, t1_e45q2fr, t1_e45qn42, t1_e465cee, t1_e471v3w, t1_e47m16x, t1_e47qnva]","[t1_e45llf2, t1_e45mtzr, t1_e45p1ij, t1_e45pmfu, t1_e45q2fr, t1_e45qn42, t1_e465cee, t1_e471v3w, t1_e47m16x, t1_e47qnva]",t1_e45llf2 | t1_e47qnva
3,t1_e45n34c,t1_e45ntcf,27,2,0.052632,1,t3_973yb1,213.50,"[t1_e45n34c, t1_e45ntcf]","[t1_e45n34c, t1_e45ntcf]",t1_e45n34c | t1_e45ntcf
4,t1_e45l4gy,t1_e475ebc,15,2,0.052632,1,t3_973yb1,83.25,"[t1_e45l4gy, t1_e45m7si, t1_e46gi63, t1_e475ebc]","[t1_e45l4gy, t1_e45m7si, t1_e46gi63, t1_e475ebc]",t1_e45l4gy | t1_e475ebc


In [5]:
# Load predicate lexicons
my_lexicon_df = pd.read_csv('my_predicate_lexicons.csv')
my_lexicon_df = my_lexicon_df[['word','lexicon category','arguments','polysemous','exclude for polysemy?','PA','IA']]
lexicon_cat_annots = my_lexicon_df.loc[(~pd.isnull(my_lexicon_df['PA'])) & 
                                       (~pd.isnull(my_lexicon_df['IA']))].copy()
lexicon_cat2PA = dict(zip(lexicon_cat_annots['lexicon category'], lexicon_cat_annots['PA']))
lexicon_cat2IA = dict(zip(lexicon_cat_annots['lexicon category'], lexicon_cat_annots['IA']))

my_lexicon_df = my_lexicon_df.loc[my_lexicon_df['exclude for polysemy?']!='y'].copy()
print(my_lexicon_df.shape)
display(my_lexicon_df.head(10))
print(my_lexicon_df['lexicon category'].value_counts())
preds_set = set(my_lexicon_df['word'].values)

mwe_preds = my_lexicon_df.loc[my_lexicon_df['word'].apply(lambda x: len(x.split( )) > 1)]['word'].values
mwe_preds_set = set(mwe_preds)
print(f'\nFound {len(mwe_preds_set)} MWE predicates.')
print(mwe_preds)

(407, 7)


,word,lexicon category,arguments,polysemous,exclude for polysemy?,PA,IA
0,call up,remember,dobj,n,NaN,high,neut
1,recall,remember,cc,n,NaN,NaN,NaN
2,recollect,remember,dobj,n,NaN,NaN,NaN
3,remember,remember,cc,n,NaN,NaN,NaN
4,think back,remember,pobj,n,NaN,NaN,NaN
5,think of,remember,dobj,n,NaN,NaN,NaN
6,leave off,forget/err,dobj,n,NaN,other,high
7,misunderstand,forget/err,dobj,n,NaN,NaN,NaN
8,omit,forget/err,dobj,n,NaN,NaN,NaN
9,miss,forget/err,dobj,n,NaN,NaN,NaN


think                       23
argue                       21
say                         19
predict                     19
be_shown                    18
arrive                      18
claim                       17
show                        16
mention                     16
arrive_via_reasoning        16
advance                     15
reason                      13
forget/err                  12
declare                     11
deny                        11
repeat                      11
advise                      10
affirm                       9
predict_sci                  9
concede                      8
explain_sci                  8
show_tentative               8
show_hidden                  8
promise                      7
high_commitment              7
agree                        7
doubt                        7
express                      7
emphasize                    7
imply                        7
remember                     6
predict_tentative            6
understa

In [ ]:
def get_subj_pred_category_counts(path_guid, subj_set, pred_set, 
                                  subj_pred_tups_per_subtree=subj_pred_tups_per_subtree,
                                  verbose=False):
    """
    Counts the number of occurrences of a (subj, pred) combination within an argument path.
    Example usage: 
        `get_subj_pred_category_counts('t1_e45l4gy | t1_e475ebc', {'you'}, WORD_CATEGORIES['concessive']`.
    """
    
    all_subj_pred_tups = subj_pred_tups_per_subtree[path_guid]
    
    if len(subj_set) == 1:
        in_category_tups = [x for x in all_subj_pred_tups
                            if (len(x['pred_neg']['text']) == 0) 
                            and (len(x['pred_q']['text'])==0) 
                            and (len(x['pred_cond']['text'])==0) 
                            and (x['in_quote']==False) 
                            and (len(set(x['pred_modal']['lemma']).intersection(disallowed_modals)) == 0) 
                            and (x['main_subj']['text'] is not None) 
                            and ((x['main_subj']['lemma'].lower() in subj_set) 
                                or (x['main_subj']['text'].lower() in subj_set))
                                #or (len(set(x['subj_mods']['lemma']).intersection(subj_set)) > 0))
                            and x['main_pred']['lemma'][0] in pred_set]
    else:
        in_category_tups = [x for x in all_subj_pred_tups
                            if (len(x['pred_neg']['text']) == 0) 
                            and (len(x['pred_q']['text'])==0) 
                            and (len(x['pred_cond']['text'])==0) 
                            and (x['in_quote']==False) 
                            and (len(set(x['pred_modal']['lemma']).intersection(disallowed_modals)) == 0) 
                            and (x['main_subj']['text'] is not None) 
                            and ((x['main_subj']['lemma'].lower() in subj_set) 
                                or (x['main_subj']['text'].lower() in subj_set)
                                or (len(set([l.lower() for l in x['subj_mods']['lemma']]).intersection(subj_set)) > 0))
                            and x['main_pred']['lemma'][0] in pred_set]
    if verbose:
        print(in_category_tups)
    return len(in_category_tups) # excludes negation, quotes, etc.